# Parse Input Data

This notebook implements data parsing and data normalisation.

In [ ]:
import lzma
import numpy as np
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


## ECB Data

ECB data are EUR yield curves and EUR-denominated FX rates.

In [ ]:
path = "../data/input/ecb/"

### FX Rates

In [ ]:
file_name = "ECB Data Portal_20240512160936_fx.csv"
data = pd.read_csv(path + file_name)
data = data.drop(["TIME PERIOD"], axis=1)
data = data.rename(
    {
        "Swiss franc/Euro (EXR.D.CHF.EUR.SP00.A)" : "CHF-EUR",
        "Chinese yuan renminbi/Euro (EXR.D.CNY.EUR.SP00.A)" : "CNY-EUR",
        "UK pound sterling/Euro (EXR.D.GBP.EUR.SP00.A)" : "GBP-EUR",
        "Japanese yen/Euro (EXR.D.JPY.EUR.SP00.A)" : "JPY-EUR",
        "US dollar/Euro (EXR.D.USD.EUR.SP00.A)" : "USD-EUR",
    },
    axis = 1,
)
data["DATE"] = pd.to_datetime(data["DATE"])
data_fx = data
data_fx

In [ ]:
fig = make_subplots(rows = 5, cols = 1, subplot_titles=("USD-EUR", "GBP-EUR", "CHF-EUR", "JPY-EUR", "CNY-EUR"))
fig.add_trace(go.Scatter(x=data["DATE"], y=data["USD-EUR"]), row=1, col=1)
fig.add_trace(go.Scatter(x=data["DATE"], y=data["GBP-EUR"]), row=2, col=1)
fig.add_trace(go.Scatter(x=data["DATE"], y=data["CHF-EUR"]), row=3, col=1)
fig.add_trace(go.Scatter(x=data["DATE"], y=data["JPY-EUR"]), row=4, col=1)
fig.add_trace(go.Scatter(x=data["DATE"], y=data["CNY-EUR"]), row=5, col=1)
fig.update_layout(height=900, width=1600, showlegend=False)

fig.show()

### Interest Rates

In [ ]:
file_name = "ECB Data Portal_20240512160755_yc.csv"
data = pd.read_csv(path + file_name)
data = data.drop(["TIME PERIOD"], axis=1)
terms = [ label.split(" ")[-1][1:-1].split("_")[-1] for label in data.columns[1:] ]
data = data.rename(dict( zip(data.columns[1:], terms)), axis = 1)
cols = [ "DATE",
    "3M", "6M", "9M", "1Y", "2Y", "3Y",
    "4Y", "5Y", "6Y", "7Y", "8Y", "9Y",
    "10Y", "12Y", "15Y", "20Y", "25Y", "30Y",
]
data = data[cols]
data["DATE"] = pd.to_datetime(data["DATE"])
data_ecb = data
data_ecb

In [ ]:
px.line(data, x="DATE", y=data.columns[1:])

## UK Bank of England Data

In [ ]:
path = "../data/input/boe/"

In [ ]:
def normalised_term(years):
    months = round(12*years)
    y = months // 12
    m = months % 12
    term = ""
    if y > 0:
        term = term + str(y) + "Y"
    if m > 0:
        term = term + str(m) + "M"
    return term

In [ ]:
def boe_data(path, file_name, sheet_name):
    data = pd.read_excel(path + file_name, sheet_name=sheet_name, header=3)
    data = data.drop(0, axis=0)
    data = data.rename({ "years:" : "DATE" }, axis=1)
    data["DATE"] = pd.to_datetime(data["DATE"])
    # terms = [ str(round(12*y)) + "M" for y in data.columns[1:]]
    terms = [ normalised_term(y) for y in data.columns[1:]]
    data = data.rename(dict( zip(data.columns[1:], terms)), axis = 1)
    return data

### Period 2000 - 2004, Short End

In [ ]:
file_name = "GLC Nominal daily data_2000 to 2004.xlsx"
sheet_name = "3. nominal spot, short end"
data_boe_1a = boe_data(path, file_name, sheet_name)
data_boe_1a

### Period 2000 - 2004, Long End

In [ ]:
file_name = "GLC Nominal daily data_2000 to 2004.xlsx"
sheet_name = "4. nominal spot curve"
data_boe_1b = boe_data(path, file_name, sheet_name)
data_boe_1b

### Period 2005 - 2015, Short End

In [ ]:
file_name = "GLC Nominal daily data_2005 to 2015.xlsx"
sheet_name = "3. spot, short end"
data_boe_2a = boe_data(path, file_name, sheet_name)
data_boe_2a

### Period 2005 - 2015, Long End

In [ ]:
file_name = "GLC Nominal daily data_2005 to 2015.xlsx"
sheet_name = "4. spot curve"
data_boe_2b = boe_data(path, file_name, sheet_name)
data_boe_2b

### Period 2016 - Present, Short End

In [ ]:
file_name = "GLC Nominal daily data_2016 to present.xlsx"
sheet_name = "3. spot, short end"
data_boe_3a = boe_data(path, file_name, sheet_name)
data_boe_3a

### Period 2016 - Present, Long End

In [ ]:
file_name = "GLC Nominal daily data_2016 to present.xlsx"
sheet_name = "4. spot curve"
data_boe_3b = boe_data(path, file_name, sheet_name)
data_boe_3b

## US Treasury Data

In [ ]:
path = "../data/input/us_treasury/"

In [ ]:
def us_data(path, file_name):
    data = pd.read_csv(path + file_name)
    data = data.rename({ "Date" : "DATE" }, axis=1)
    terms = [ label.replace(" ", "")[0:-1] for label in data.columns[1:] ]
    data = data.rename(dict( zip(data.columns[1:], terms)), axis = 1)
    try:
        data["DATE"] = pd.to_datetime(data["DATE"], format="%m/%d/%y")
    except ValueError:
        data["DATE"] = pd.to_datetime(data["DATE"], format="%m/%d/%Y")
    return data    

In [ ]:
file_name = "yield-curve-rates-1990-2023.csv"
data_us_1 = us_data(path, file_name)
data_us_1

In [ ]:
file_name = "daily-treasury-rates.csv"
data_us_2 = us_data(path, file_name)
data_us_2

# Data Selection and Data Normalisation

In this section, we specify data selection and data normalisation. This is particularly relevant for yield curve data.

**Yield Curve Terms**

We select data for the following terms:

3M,   6M,  9M,  1Y,  2Y,  3Y,
4Y,   5Y,  6Y,  7Y,  8Y,  9Y,
10Y, 12Y, 15Y, 20Y, 25Y, 30Y.

If a term is not available for a data set then it is skipped.

**Date Selection**

We select all dates with available data from the data sets.

Data set entries with *null* data are excluded.

**Data Format**

We store individual data points with the following properties (columns):
  - DATE, observation date in format YYYY-MM-DD.
  - CURRENCY, market currency in format CUR or currency pair in format FOR-DOM.
  - TERM, maturity term as months or years, formatted as string, e.g. 6M, 2Y; empty for FX rates.
  - MONTHS, maturity term in months; 0 for FX rates.
  - VALUE, annual interest rate as decimal number with continuous compounding, FX rates as EUR-denominated price.
  - DESCR, free text string, description of source and value format.
 

In [ ]:
terms = [
     "3M",  "6M", "9M",   "1Y",  "2Y",  "3Y",
     "4Y",  "5Y",  "6Y",  "7Y",  "8Y",  "9Y",
    "10Y", "12Y", "15Y", "20Y", "25Y", "30Y",
]

columns = ["DATE", "CURRENCY", "TERM", "MONTHS", "VALUE", "DESC"]

In [ ]:
def months_from_term(term):
    val = int(term[0:-1])
    unit = term[-1:]
    if unit == "M":
        return val
    if unit == "Y":
        return 12 * val

## FX Rates

In [ ]:
data = data_fx.melt("DATE", var_name="CURRENCY", value_name="VALUE")
data["TERM"] = ""
data["MONTHS"] = 0
data["DESC"] = "ECB Data Portal"
data = data[columns]
data_set_fx = data
data_set_fx

## EUR Rates

In [ ]:
data_list = []
for term in terms:
    # print(term, month)
    month = months_from_term(term)
    data = pd.DataFrame()
    data["DATE"] = data_ecb["DATE"]
    data["CURRENCY"] = "EUR"
    data["TERM"] = term
    data["MONTHS"] = month
    data["VALUE"] = data_ecb[term] * 1.0e-2
    data["DESC"] = "ECB Data Portal, continuous compounding, Act/365 (Fixed)"
    data_list.append(data)

data_set_ecb = pd.concat(data_list, ignore_index=True)
data_set_ecb

## GBP Rates

In [ ]:
data_boe_a = pd.concat([data_boe_1a, data_boe_2a, data_boe_3a], ignore_index=True)
data_boe_b = pd.concat([data_boe_1b, data_boe_2b, data_boe_3b], ignore_index=True)
dt = [ dt.days for dt in (data_boe_a["DATE"] - data_boe_b["DATE"]) ]
assert np.max(dt) == 0
assert np.min(dt) == 0

In [ ]:
data_list = []
for term in terms:
    month = months_from_term(term)
    # print(term, month)
    data = pd.DataFrame()
    data["DATE"] = data_boe_b["DATE"]
    data["CURRENCY"] = "GBP"
    data["TERM"] = term
    data["MONTHS"] = month
    if month < 12:
        data["VALUE"] = data_boe_a[term] * 1.0e-2
    else:
        data["VALUE"] = data_boe_b[term] * 1.0e-2
    data["DESC"] = "BoE Statistics Yield Curve, continuous compounding, Act/365 (Fixed)"
    data_list.append(data)

data_set_boe = pd.concat(data_list, ignore_index=True)
data_set_boe

## USD Rates

In [ ]:
data_us = pd.concat([data_us_2, data_us_1], ignore_index=True)

data_list = []
for term in terms:
    if not term in data_us.columns:
        continue
    month = months_from_term(term)
    # print(term, month)
    data = pd.DataFrame()
    data["DATE"] = data_us["DATE"]
    data["CURRENCY"] = "USD"
    data["TERM"] = term
    data["MONTHS"] = month
    rates = data_us[term] * 1.0e-2
    if month <= 6:
        cont_rates = np.log(1.0 + rates * month / 12.0) / (month / 12.0)
    else:
        cont_rates = np.log(np.power(1.0 + rates/2.0, 2 * month / 12)) / (month / 12.0)
    data["VALUE"] = cont_rates
    data["DESC"] = "US Treasury Par Yield Curve Rates, continuous compounding, Act/365 (Fixed)"
    data_list.append(data)

data_set_us = pd.concat(data_list, ignore_index=True)
data_set_us

## Consolidated Data Set

In [ ]:
data_set = pd.concat([ data_set_fx, data_set_ecb, data_set_boe, data_set_us ], ignore_index = True)
data_set = data_set.dropna()
data_set = data_set.sort_values(["CURRENCY", "MONTHS", "DATE"], ignore_index=True)
data_set

### Data Analysis

In [ ]:
from describe_data_set import describe_values, describe_dates

In [ ]:
describe_values(data_set)

In [ ]:
describe_dates(data_set)

### Data Export 

In [ ]:
path = "../data/consolidated/"
file_name = "data_set_full.csv"

data_set.to_csv(path + file_name, index=False, float_format="%.6f")

We also zip the output CSV file to store it efficiently in the repository.

In [ ]:
with open(path + file_name, "rb") as f:
    data = f.read()

with lzma.open(path + file_name + ".xz", "w") as f:
    f.write(data)